<div align="center"><p style="font-family: 'Mochiy Pop P One';font-size:32px; font-weight: bold; color:teal" id="top">Equity Market Crisis Regime Prediction using Machine Learning GBDT </p></div>

<div align="center"><p style="font-family: 'Mochiy Pop P One';font-size:28px; font-weight: bold; color:teal">Part VIII: Deployment of LightGBM Models (end-to-end process)</p>

<p style="font-family: 'Mochiy Pop P One';font-size:28px; font-weight: bold; color:black"><mark>█This notebook runs daily to produce predictions█ </mark></p>
    
<p style="font-family: 'Mochiy Pop P One';font-size:20px;  font-weight: bold; color:mediumblue; ">
This notebook is the final part of 8-piece series. Here is the quick access to the whole series.
    
</p> 
<br>

</div>

<div align="left" style="font-family: 'Mochiy Pop P One';font-size:16px; font-weight: bold; padding:10px; background:#fafafa; margin-left:2px">
    
<ol>     
<li><a href="https://www.kaggle.com/code/xxxxyyyy80008/equity-market-crisis-regime-prediction-using-gbdt1">Part I: Introduction</a></li>
<li><a href="https://www.kaggle.com/code/xxxxyyyy80008/equity-market-crisis-regime-prediction-using-gbdt2">Part II: Data Preparation</a>  </li>
<li><a href="https://www.kaggle.com/code/xxxxyyyy80008/equity-market-crisis-regime-prediction-using-gbdt3">Part III: Exploratory data analysis</a>  </li> 
<li><a href="https://www.kaggle.com/code/xxxxyyyy80008/equity-market-crisis-regime-prediction-using-gbdt4">Part IV: Feature Selection, Hyperparameter Tuning (LightGBM)</a>  </li>     
<li><a href="https://www.kaggle.com/code/xxxxyyyy80008/equity-market-crisis-regime-prediction-using-gbdt5">Part V: Model Evaluation and Interpretation (LightGBM)</a>  </li>    
<li><a href="https://www.kaggle.com/code/xxxxyyyy80008/equity-market-crisis-regime-prediction-using-gbdt6">Part VI: SVM and Neural Networks (MLP and 1D-CNN)</a>  </li>   
<li><a href="https://www.kaggle.com/code/xxxxyyyy80008/equity-market-crisis-regime-prediction-using-gbdt7">Part VII: Compare GBDT Models: XGBoost and LightGBM</a>  </li>     
<li><a href="https://www.kaggle.com/code/xxxxyyyy80008/equity-market-crisis-regime-prediction-using-gbdt8">Part VIII: Deployment of LightGBM Models (end-to-end process)</a>  </li>
<li><mark><a href="https://www.kaggle.com/code/xxxxyyyy80008/predict-stock-market-crashes">Predict Stock Market Crashes</a></mark>  </li>
</ol>        
    
</div>



<p style="font-family: 'Mochiy Pop P One';font-size:14px;  font-weight: bold; color:forestgreen; font-style:italic" id="toc">Quick Navigation Of This Notebook: </p>
<div align="left" style="font-family: 'Mochiy Pop P One';font-size:13px; font-weight: bold; color: teal">
<span style="padding:10px;background:wheat;margin-left:2px"> <a href=#1 stype="color: teal">1. Install and Load Packages</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#2 stype="color: teal">2. Data Scraping and Processing</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#3 stype="color: teal">3. Feature Engineering</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#4 stype="color: teal">4. Train LightGBM Models</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#5 stype="color: teal">5. Predict</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#6 stype="color: teal">6. Today's Prediction</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#7 stype="color: teal">7. Explainable ML</a> </span>
</div>


<a id="1"></a>

<p style="font-family: 'Mochiy Pop P One';font-size:20px;  font-weight: bold; color:teal">1. Install and Load Packages </p>





In [ ]:
import pandas as pd
import numpy as np
import os
import gc
import copy
from pathlib import Path
from datetime import datetime, timedelta, date
import time
from dateutil.relativedelta import relativedelta 

import pyarrow.parquet as pq
import pyarrow as pa

from tqdm import tqdm

In [ ]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

pd.options.display.float_format = '{:,.2f}'.format

import warnings
warnings.filterwarnings("ignore")

import pytorch_lightning as pl
random_seed=1
pl.seed_everything(random_seed)

In [ ]:
!pip install mplfinance==0.12.9b1
!pip install yfinance==0.1.66
!pip install fredapi


<p style="font-family: 'Mochiy Pop P One';font-size:20px;  font-weight: bold; color:teal" id="2">2. Data Scraping and Processing </p>


<p style="font-family: 'Mochiy Pop P One';font-size:14px;  font-weight: bold; color:forestgreen; font-style:italic" id="toc">Quick Navigation Of This Notebook: </p>
<div align="left" style="font-family: 'Mochiy Pop P One';font-size:13px; font-weight: bold; color: teal">
<span style="padding:10px;background:wheat;margin-left:2px"> <a href=#top stype="color: teal">BACK TO TOP</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"> <a href=#1 stype="color: teal">1. Install and Load Packages</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#2 stype="color: teal">2. ...</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#3 stype="color: teal">3. Feature Engineering</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#4 stype="color: teal">4. Train LightGBM Models</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#5 stype="color: teal">5. Predict</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#6 stype="color: teal">6. Today's Prediction</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#7 stype="color: teal">7. Explainable ML</a> </span>
</div>


<p style="font-family: 'Mochiy Pop P One';font-size:14px;  font-weight: bold; color:forestgreen; font-style:italic" id="toc">Section Navigation: </p>
<div align="left" style="font-family: 'Mochiy Pop P One';font-size:13px; font-weight: bold; color: teal">
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2.0 stype="color: teal">2.0. Prepare Tickers</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2.1 stype="color: teal">2.1. Download Data From Yahoo Finance</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.2 stype="color: teal">2.2. Download Data From FRED</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.3 stype="color: teal">2.3. Target Engineering</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.4 stype="color: teal">2.4. Data Processing</a> </span>
</div>

<br>  


<p style="font-family: 'Mochiy Pop P One';font-size:18px;  font-weight: bold; color:darkblue" id="2.0">2.0. Prepare Tickers  <span style="padding:10px;background:wheat;margin-left:2px"> <a href=#top stype="color: teal">BACK TO TOP</a> </span> </p>



In [ ]:
final_feats = ['NASDAQ100_std125_chg200', 'MICH_ma_dist20_50', 'STLPPM_ma_dist20_50', 'GC=F_chg5', 'STLFSI3_pct_chg5', 'VIXCLS_chg20', 'VIX_zscore250', 'IRLTLT01JPM156N_std125', 'LFWA64TTJPM647S_volat60', 'WILLLRGCAPVAL_std60_chg200', 'GC=F_pct_chg20_above200dMA', 'PAYNSA_zscore200', 'T1YFF_pct_chg20', 'NIKKEI225_zscore120', 'LRUN64TTJPM156S_std125_chg200', 'T1YFF_ma_dist200_50', 'WILLSMLCAP_chg20', 'HSI_chg5', 'JPNASSETS_zscore120', 'NIKKEI225_std60_chg200', 'HSI_chg20', 'N225_pct_chg20', 'FVX_chg5', 'NASDAQ100_pct_chg10', 'RBJPBIS_ma_dist20_50', 'KS11_pct_chg5', 'WILLSMLCAP_chg10', 'T10Y2Y_pct_chg120', 'JTSOSL_std60_chg200', 'BAMLHYH0A0HYM2TRIV_std125_chg200', 'IRLTLT01JPM156N_std60_chg200', 'TB3SMFFM_zscore200', 'NIKKEI225_pct_chg10', 'RBJPBIS_zscore60', 'WILLREITIND_ma_dist20_50', 'LRUN64TTJPM156S_pct_chg250', 'MICH_pct_chg5', 'TNX_mean_std120', 'BAMLHE00EHYITRIV_chg10', 'VXNCLS_mean_std120', 'MYAGM2JPM189S_std125', 'BAA10YM_zscore120', 'T10Y3M_chg250', 'MYAGM2JPM189S_chg20', 'EXPINF1YR_chg5_above200dMA', 'NFCICREDIT_pct_chg60', 'GC=F_pct_chg10', 'REAINTRATREARAT1YE_pct_chg120', 'NDX_chg120', 'GVZCLS_chg5', 'KCFSI_chg20', 'AAA10Y_pct_chg5', 'TENEXPCHAINFRISPRE_ma_dist20_50', 'T5YIEM_pct_chg20', 'BAMLHE00EHYITRIV_chg5', 'USSLIND_pct_chg120', 'FVX_pct_chg10', 'T10YIEM_chg60', 'WILLREITIND_pct_chg20', 'BAA10Y_pct_chg10', 'T10YFF_chg10', 'LFWA64TTJPM647S_chg60', 'OVXCLS_volat60', 'RUT_pct_chg10', 'VXNCLS_chg10', 'BAA10Y_zscore250', 'NDX_chg10', 'T10Y2YM_zscore250', 'JPNASSETS', 'CPFF_pct_chg10', 'WILLREITIND_chg20', 'TNX_chg20', 'IRSTCI01JPM156N_chg20', 'GC=F_zscore60', 'CPFF_pct_chg250', 'CPFF_chg5', 'BAMLHYH0A3CMTRIV_chg10', 'CPFF_pct_chg5', 'VXNCLS_chg5', 'GC=F_chg20', 'FVX_pct_chg5', 'T10Y2Y_pct_chg250', 'FTSE_chg10', 'N100_std60_chg200', 'HSI_zscore60', 'NFCIRISK_pct_chg60', 'STLFSI3_pct_chg60', 'VIX_pct_chg10', 'RVXCLS_chg10', 'NFCICREDIT_pct_chg5', 'STLPPM_chg5', 'T3MFF_zscore60', 'ABCOMP_chg5', 'T10Y3M_pct_chg250', 'ABCOMP_pct_chg5', 'USSLIND_pct_chg20', 'STLPPM_pct_chg5', 'T3MFF_chg10', 'FVX_zscore200', 'VXDCLS_chg5', 'VIX_zscore60', 'VIX_chg20', 'STOXX50E_pct_chg10', 'DJI_zscore120', 'STLFSI3_chg5', 'NFCICREDIT', 'T10YIEM_pct_chg60', 'NDX_zscore60', 'HSI_chg10', 'AAAFF_pct_chg5', 'FTSE_mean_std120', 'MYAGM2JPM189S_chg5', 'BAMLHE00EHYITRIV_pct_chg250', 'REAINTRATREARAT1MO_ma_dist500_250', 'CPFF_pct_chg20', 'VXDCLS_pct_chg10', 'BAMLHYH0A3CMTRIV_zscore120', 'NFCI_chg5', 'FTSE_chg20', 'FVX_ma_dist20_50', 'T10YFF_pct_chg20', 'NFCILEVERAGE_chg250', 'BAMLHYH0A0HYM2TRIV_pct_chg10', 'T10Y2YM_std125_chg200', 'CFNAIDIFF_std60', 'N100_zscore120', 'DJI_chg10']

print(final_feats)
print(len(final_feats))

In [ ]:
raw_feats = []
for feat in final_feats:
    raw_feats.append(feat.split('_')[0])
    
raw_feats = list(set(raw_feats))

In [ ]:
print(len(raw_feats), raw_feats)

In [ ]:
#https://sg.finance.yahoo.com/world-indices/
#https://sg.finance.yahoo.com/bonds/
#https://sg.finance.yahoo.com/commodities/
benchmark_tickers = ['^GSPC', '^DJI', '^IXIC', '^RUT', '^VIX', 
                     '^TYX', '^TNX','^FVX', '^IRX','^NDX', 'DX-Y.NYB',  'EEM','IEMG','^FTSE','^STOXX50E',
                     '^N225', 'JPY=X', '^KS11', 'GC=F', 'CL=F', '^HSI', '^KLSE', '^N100'
                    ]#'^SPX'

In [ ]:
yfinance_tickers = ['^GSPC']
fed_tickers = []
for f in raw_feats:
    if (f'^{f}' in benchmark_tickers):
        yfinance_tickers.append(f'^{f}')
    elif (f'{f}' in benchmark_tickers):
        yfinance_tickers.append(f'{f}')
    else:
        fed_tickers.append(f)
        
print(yfinance_tickers)
print(fed_tickers)


<p style="font-family: 'Mochiy Pop P One';font-size:18px;  font-weight: bold; color:darkblue" id="2.1">2.1. Download Data From Yahoo Finance <span style="padding:10px;background:wheat;margin-left:2px"> <a href=#top stype="color: teal">BACK TO TOP</a> </span></p>



<p style="font-family: 'Mochiy Pop P One';font-size:14px;  font-weight: bold; color:forestgreen; font-style:italic" id="toc">Section Navigation: </p>
<div align="left" style="font-family: 'Mochiy Pop P One';font-size:13px; font-weight: bold; color: teal">
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2 stype="color: teal">BACK TO SECTION TOP</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2.0 stype="color: teal">2.0. Prepare Tickers</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2.1 stype="color: teal">2.1. Download Data From Yahoo Finance</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.2 stype="color: teal">2.2. Download Data From FRED</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.3 stype="color: teal">2.3. Target Engineering</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.4 stype="color: teal">2.4. Data Processing</a> </span>
</div>

<br>  

In [ ]:
import yfinance as yf

In [ ]:
#https://github.com/ranaroussi/yfinance/blob/main/yfinance/base.py
#     def history(self, period="1mo", interval="1d",
#                 start=None, end=None, prepost=False, actions=True,
#                 auto_adjust=True, back_adjust=False,
#                 proxy=None, rounding=False, tz=None, timeout=None, **kwargs):

dfs = {}

for ticker in  yfinance_tickers:
    cur_data = yf.Ticker(ticker)
    hist = cur_data.history(period="max")
    print(ticker, datetime.now(), ticker, hist.shape, hist.index.min(), hist.index.max())

    dfs[ticker] = hist
    



<p style="font-family: 'Mochiy Pop P One';font-size:18px;  font-weight: bold; color:darkblue" id="2.3">2.3. Target Engineering  <span style="padding:10px;background:wheat;margin-left:2px"> <a href=#top stype="color: teal">BACK TO TOP</a> </span></p>



<p style="font-family: 'Mochiy Pop P One';font-size:14px;  font-weight: bold; color:forestgreen; font-style:italic" id="toc">Section Navigation: </p>
<div align="left" style="font-family: 'Mochiy Pop P One';font-size:13px; font-weight: bold; color: teal">
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2 stype="color: teal">BACK TO SECTION TOP</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2.0 stype="color: teal">2.0. Prepare Tickers</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2.1 stype="color: teal">2.1. Download Data From Yahoo Finance</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.2 stype="color: teal">2.2. Download Data From FRED</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.3 stype="color: teal">2.3. Target Engineering</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.4 stype="color: teal">2.4. Data Processing</a> </span>
</div>

<br>  

In [ ]:
df_target = dfs['^GSPC'].copy(deep=True)


In [ ]:
#15day_pct change
df_target['pct_chg15'] = df_target['Close'].pct_change(15)
df_target['regime_change'] = df_target['pct_chg15'].rolling(window=5000, min_periods=2000).apply(lambda x:x[:-1].quantile(q=0.05)>x[-1])
#in the papers, the authors did not describe shifting the target. 
#here shift the target by n days to use X on d0 to predict if there will be a crisis regime in n days
df_target['target'] = df_target['regime_change'].shift(-3)

In [ ]:
df_target.tail(5)

In [ ]:
df_target[['regime_change', 'target']].tail(100).plot(figsize=(16, 5))

In [ ]:
max_date = df_target.index.max()
max_date


<p style="font-family: 'Mochiy Pop P One';font-size:18px;  font-weight: bold; color:darkblue" id="2.2">2.2. Download Data From FRED  <span style="padding:10px;background:wheat;margin-left:2px"> <a href=#top stype="color: teal">BACK TO TOP</a> </span></p>



<p style="font-family: 'Mochiy Pop P One';font-size:14px;  font-weight: bold; color:forestgreen; font-style:italic" id="toc">Section Navigation: </p>
<div align="left" style="font-family: 'Mochiy Pop P One';font-size:13px; font-weight: bold; color: teal">
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2 stype="color: teal">BACK TO SECTION TOP</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2.0 stype="color: teal">2.0. Prepare Tickers</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2.1 stype="color: teal">2.1. Download Data From Yahoo Finance</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.2 stype="color: teal">2.2. Download Data From FRED</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.3 stype="color: teal">2.3. Target Engineering</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.4 stype="color: teal">2.4. Data Processing</a> </span>
</div>

<br>  

In [ ]:
# https://www.multpl.com/s-p-500-dividend-yield/table/by-month
# https://www.multpl.com/s-p-500-earnings
# https://www.multpl.com/s-p-500-pe-ratio/table/by-month
# https://www.multpl.com/shiller-pe/table/by-year
#https://fred.stlouisfed.org/categories/46

#https://fredaccount.stlouisfed.org/apikeys
#https://mortada.net/python-api-for-fred.html
#https://github.com/mortada/fredapi
#https://www.briancjenkins.com/fredpy/docs/build/html/fredpy_examples.html
from fredapi import Fred
fred = Fred('c8d4f8e608b42f6d85dd037f2080e451')

In [ ]:
#dir(fred)

In [ ]:
#help(fred.get_series)

In [ ]:
%%time
desc_dfs = []
for series_id in tqdm(fed_tickers):
    desc_dfs.append(fred.get_series_info(series_id))
    

In [ ]:
df_desc = pd.concat(desc_dfs, axis=1).T
df_desc['popularity'] = df_desc['popularity'].astype(int)
df_desc['discontinued'] = df_desc['title'].apply(lambda x: 'DISCONTINUED' in x)
df_desc.sort_values(by='popularity', ascending=False, inplace=True)

In [ ]:
df_desc.index=df_desc['id']
df_desc.index.name='series_id'

df_desc['last_updated'] = pd.to_datetime(df_desc['last_updated'].values)

<p style="font-family: 'Mochiy Pop P One';font-size:20px; font-weight: bold; color:black"><mark>!!!! Make sure all the fed_tickers can be retrieved via API </mark></p>


<p style="font-family: 'Mochiy Pop P One';font-size:18px; font-weight: bold; color:black">this is validated by comparing <mark>df_desc.shape[0]</mark> and <mark>len(fed_tickers)</mark></p>

In [ ]:
df_desc.shape, len(fed_tickers)

In [ ]:
from tabulate import tabulate
print(tabulate(df_desc[['id', 'title', 'frequency_short' ]], 
               headers=['id', 'title', 'frequency_short' ], tablefmt="github", showindex=False))


<p style="font-family: 'Mochiy Pop P One';font-size:20px; font-weight: bold; color:black"><mark>!!!! Pay attention to discontinued datasets </mark></p>


<p style="font-family: 'Mochiy Pop P One';font-size:18px; font-weight: bold; color:black">The output of<mark>df_desc['discontinued'].sum()</mark> should be <mark>0</mark></p>

In [ ]:
df_desc['discontinued'].sum()


<p style="font-family: 'Mochiy Pop P One';font-size:18px;  font-weight: bold; color:darkblue" id="2.4">2.4. Data Processing <span style="padding:10px;background:wheat;margin-left:2px"> <a href=#top stype="color: teal">BACK TO TOP</a> </span> </p>



<p style="font-family: 'Mochiy Pop P One';font-size:14px;  font-weight: bold; color:forestgreen; font-style:italic" id="toc">Section Navigation: </p>
<div align="left" style="font-family: 'Mochiy Pop P One';font-size:13px; font-weight: bold; color: teal">
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2 stype="color: teal">BACK TO SECTION TOP</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2.0 stype="color: teal">2.0. Prepare Tickers</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2.1 stype="color: teal">2.1. Download Data From Yahoo Finance</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.2 stype="color: teal">2.2. Download Data From FRED</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.3 stype="color: teal">2.3. Target Engineering</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.4 stype="color: teal">2.4. Data Processing</a> </span>
</div>

<br>  


Process stlouisfed data
    
For `Today`:  Grab data - most recent data will be yesterday's data updated at day-end - to predict if crisis regime is going to happen in ? days

- For `daily` frequency data: check the difference (in days) between last_updated and max_date (from target data), and make the x_date shift by n
    - if n<1, day-end update. x_date is the date 
    - if n>=1: shift 
- For `weekly` and `monthly` data:
    - convert to daily
    - shift the date to update date
    


In [ ]:
df_desc['frequency'].value_counts()

In [ ]:
df_desc['frequency_short'].value_counts()

In [ ]:
start_date = '2000-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')
end_date

In [ ]:
%%time
fed_data_dict = {}
for series_id, row in df_desc.iterrows():
    data = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
    fed_data_dict[series_id] = data
    

In [ ]:
for series_id, data in fed_data_dict.items():
    print(series_id)
    display(data.tail(5))

In [ ]:
fed_dfs = {}
for series_id, data in fed_data_dict.items():
    row = df_desc.loc[series_id]
    #n_diff = (datetime.now(tz=row['last_updated'].tzinfo) - row['last_updated']).days
    #n_diff = (max_date - datetime.strptime(row['last_updated'].strftime('%Y-%m-%d'),'%Y-%m-%d')).days
    
    #process daily data
    if row['frequency_short']=='D': 
        #n_diff = (max_date - datetime.strptime(data.index.max().strftime('%Y-%m-%d'),'%Y-%m-%d')).days
        n_diff = df_target[df_target.index>data.index.max()].shape[0]
        data.index.name='date'
        data.name=series_id
        data = data.to_frame().reset_index()
        print(row['frequency_short'],series_id, row['last_updated'], n_diff)
        
        if n_diff<1:
            data['x_date'] = data['date']
        else:
            #data['x_date'] = data['date'].shift(-n_diff)
            data['x_date'] = data['date'].shift(-n_diff).tolist()[:-n_diff] + df_target.index[-n_diff:].tolist()
        fed_dfs[series_id] = data
    else: #monthly/weekly data
        data.index.name='date'
        data.name=series_id
        data = data.to_frame().reset_index()
        
        n_diff = (datetime.strptime(row['last_updated'].strftime('%Y-%m-%d'),'%Y-%m-%d') - data['date'].max()).days 
        data['x_date'] = data['date'].apply(lambda x: x+relativedelta(days=n_diff))
        
        data_ = pd.DataFrame({'x_date':pd.date_range(start=data['x_date'].min(), end=data['x_date'].max(), freq='D')})
        data_ = data_.merge(data,  on='x_date', how='left')
        data_.fillna(method='ffill', inplace=True)
        
        print(row['frequency_short'],series_id, row['last_updated'], n_diff)
        fed_dfs[series_id] = data_

<p style="font-family: 'Mochiy Pop P One';font-size:14px;  font-weight: bold; color:forestgreen; font-style:italic" id="toc">Section Navigation: </p>
<div align="left" style="font-family: 'Mochiy Pop P One';font-size:13px; font-weight: bold; color: teal">
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2 stype="color: teal">BACK TO SECTION TOP</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2.0 stype="color: teal">2.0. Prepare Tickers</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"> <a href=#2.1 stype="color: teal">2.1. Download Data From Yahoo Finance</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.2 stype="color: teal">2.2. Download Data From FRED</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.3 stype="color: teal">2.3. Target Engineering</a> </span>
<span style="padding:10px;background:paleturquoise;margin-left:2px"><a href=#2.4 stype="color: teal">2.4. Data Processing</a> </span>
</div>

<br>  


<p style="font-family: 'Mochiy Pop P One';font-size:20px; font-weight: bold; color:mediumblue">Combine target variable, yfinance data, and fred.stlouisfed.org data</p>


In [ ]:
df = df_target[['pct_chg15', 'regime_change', 'target']].copy(deep=True)#'^GSPC'

In [ ]:
for ticker, data in dfs.items():
    df = df.merge(data[['Close']], left_index=True, right_index=True, how='left')
    df.rename(columns={'Close': ticker.replace('^', '')}, inplace=True)
df.index.name='x_date'
df.reset_index(drop=False, inplace=True)

In [ ]:
for c,data in fed_dfs.items():
    
    t = data['x_date'].value_counts()[data['x_date'].value_counts()>1]
    if t.shape[0]>0:
        print(c, data.shape)
        display(data['x_date'].value_counts()[data['x_date'].value_counts()>1])

In [ ]:
for c,data in fed_dfs.items():
    print(c, data.shape)
    df = df.merge(data[['x_date', c]], on='x_date', how='left')
    

In [ ]:
df = df[df['x_date']>='2000-01-01'].copy(deep=True)

In [ ]:
df.tail(5)

In [ ]:
df.isna().sum().sort_values().tail(100)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.tail(5)

In [ ]:
df.set_index(keys=['x_date'], inplace=True)
df.sort_index(ascending=True, inplace=True)

In [ ]:
df.tail(5)


<p style="font-family: 'Mochiy Pop P One';font-size:20px; font-weight: bold; color:mediumblue">Impute missing values</p>



In [ ]:
# print(df.shape)
# df.dropna(how='any', subset=['target'], inplace=True)
# print(df.shape)

In [ ]:
df.replace([-np.inf, np.inf], np.nan, inplace=True)
df.isna().sum()[df.isna().sum()>0]

In [ ]:
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

In [ ]:
df.isna().sum()[df.isna().sum()>0]

In [ ]:
df.tail(5)

<a id="3"></a>

<p style="font-family: 'Mochiy Pop P One';font-size:20px;  font-weight: bold; color:teal">3. Feature Engineering </p>


<p style="font-family: 'Mochiy Pop P One';font-size:14px;  font-weight: bold; color:forestgreen; font-style:italic" id="toc">Quick Navigation Of This Notebook: </p>
<div align="left" style="font-family: 'Mochiy Pop P One';font-size:13px; font-weight: bold; color: teal">
<span style="padding:10px;background:wheat;margin-left:2px"> <a href=#toc stype="color: teal">BACK TO TOP</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"> <a href=#1 stype="color: teal">1. Install and Load Packages</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#2 stype="color: teal">2. Data Scraping and Processing</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#3 stype="color: teal">3. ...</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#4 stype="color: teal">4. Train LightGBM Models</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#5 stype="color: teal">5. Predict</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#6 stype="color: teal">6. Today's Prediction</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#7 stype="color: teal">7. Explainable ML</a> </span>
</div>


In [ ]:
#calculate volatility
def cal_vol(series, period=10, eps=1e-8):
    series = series.fillna(method='ffill').fillna(method='bfill').fillna(0)

    #ensure the input data is positive numeric values
    if series.min()<0:
        return None
    
    vol = np.log(series + eps).diff(1).rolling(period).apply(lambda x: np.sqrt(np.sum(x**2)))
    
    return pd.Series(vol, name= f"vol{period}")

#calculate sharpe ratio, here assume risk free rate is 0
def cal_sharpe(series):
    series = series.fillna(method='ffill').fillna(method='bfill').fillna(0)
    
    diff_ = series.diff()
    std_ = np.std(diff_, ddof = 1)
    
    if np.isnan(std_) or std_==0:
        sharpe  = 0.0
    else:
        sharpe = np.divide(diff_.mean(), std_)
        
    return sharpe


def cal_zscore(series):
    series = series.fillna(method='ffill').fillna(method='bfill').fillna(0)
    series = np.nan_to_num(series, 0)
    m_ = series.mean()
    s_ = series.std()
    z = (series[-1] - m_)/s_
    return z

def above_mean(series):
    series = series.fillna(method='ffill').fillna(method='bfill').fillna(0)
    series = np.nan_to_num(series, 0)
    return series[:-1].mean()<series[-1]

In [ ]:
def generate_features(data_df, name_pref, feat):
    for period in [10, 20, 60, 120, 200, 250 ]:
        data_df[f'{name_pref}_chg{period}'] = data_df[feat].diff(period)
        data_df[f'{name_pref}_pct_chg{period}'] = data_df[feat].pct_change(period)
        data_df[f'{name_pref}_volat{period}'] = cal_vol(data_df[feat], period=period, eps=1e-8)
        
#         for p2 in [20, 60]:
#             data_df[f'{name_pref}_chg{period}_ma{p2}'] = data_df[f'{name_pref}_chg{period}'].rolling(window=p2, min_periods=p2).mean()
#             data_df[f'{name_pref}_pct_chg{period}_ma{p2}'] = data_df[f'{name_pref}_pct_chg{period}'].rolling(window=p2, min_periods=p2).mean()
#             data_df[f'{name_pref}_volat{period}_ma{p2}'] = data_df[f'{name_pref}_volat{period}'].rolling(window=p2, min_periods=p2).mean()

        data_df[f'{name_pref}_chg{period}_above200dMA'] = data_df[f'{name_pref}_chg{period}'].fillna(0).rolling(window=200, min_periods=200).apply(above_mean)
        data_df[f'{name_pref}_pct_chg{period}_above200dMA'] = data_df[f'{name_pref}_pct_chg{period}'].fillna(0).rolling(window=200, min_periods=200).apply(above_mean)
        data_df[f'{name_pref}_volat{period}_above200dMA'] = data_df[f'{name_pref}_volat{period}'].fillna(0).rolling(window=200, min_periods=200).apply(above_mean)
    
    data_df[f'{name_pref}_ma_dist20_50'] = data_df[feat].rolling(window=20, min_periods=20).mean() - data_df[feat].rolling(window=50, min_periods=50).mean()
    data_df[f'{name_pref}_ma_dist200_50'] = data_df[feat].rolling(window=200, min_periods=200).mean() - data_df[feat].rolling(window=50, min_periods=50).mean()
    
    for period in [60, 120, 200, 250]:
        data_df[f'{name_pref}_mean_std{period}'] = data_df[feat].rolling(window=period, min_periods=period).apply(cal_sharpe)
        data_df[f'{name_pref}_mean_std{period}_above200dMA'] = data_df[f'{name_pref}_mean_std{period}'].rolling(window=200, min_periods=200).apply(above_mean)
        data_df[f'{name_pref}_zscore{period}'] = data_df[feat].rolling(window=period, min_periods=period).apply(cal_zscore)
        data_df[f'{name_pref}_zscore{period}_above200dMA'] = data_df[f'{name_pref}_zscore{period}'].rolling(window=200, min_periods=200).apply(above_mean)
        data_df[f'{name_pref}_ma_dist20_50_zscore{period}'] = data_df[f'{name_pref}_ma_dist20_50'].rolling(window=period, min_periods=period).apply(cal_zscore)
        data_df[f'{name_pref}_ma_dist200_50_zscore{period}'] = data_df[f'{name_pref}_ma_dist200_50'].rolling(window=period, min_periods=period).apply(cal_zscore)
       


In [ ]:
def generate_features(data_df, name_pref, feat):
    for period in [5, 10, 20, 60, 120, 250 ]:
        data_df[f'{name_pref}_chg{period}'] = data_df[feat].diff(period)
        data_df[f'{name_pref}_pct_chg{period}'] = data_df[feat].pct_change(period)
        
        if period in [5, 20]:
            data_df[f'{name_pref}_chg{period}_above200dMA'] = data_df[f'{name_pref}_chg{period}'].fillna(0).rolling(window=200, min_periods=200).apply(above_mean)
            data_df[f'{name_pref}_pct_chg{period}_above200dMA'] = data_df[f'{name_pref}_pct_chg{period}'].fillna(0).rolling(window=200, min_periods=200).apply(above_mean)
        
    data_df[f'{name_pref}_ma_dist20_50'] = data_df[feat].rolling(window=20, min_periods=20).mean() - data_df[feat].rolling(window=50, min_periods=50).mean()
    data_df[f'{name_pref}_ma_dist200_50'] = data_df[feat].rolling(window=200, min_periods=200).mean() - data_df[feat].rolling(window=50, min_periods=50).mean()
    data_df[f'{name_pref}_ma_dist500_250'] = data_df[feat].rolling(window=500, min_periods=500).mean() - data_df[feat].rolling(window=250, min_periods=250).mean()
    
    for period in [60, 125,]:
        data_df[f'{name_pref}_std{period}'] = data_df[feat].rolling(window=period, min_periods=period).std()
        data_df[f'{name_pref}_std{period}_chg200'] = data_df[f'{name_pref}_std{period}'].diff(200)
        
        if data_df[feat].min()>0:
            data_df[f'{name_pref}_volat{period}'] = cal_vol(data_df[feat], period=period, eps=1e-8)
        
        
    for period in [120, 250]:
        if data_df[feat].min()>0:
            data_df[f'{name_pref}_mean_std{period}'] = data_df[feat].rolling(window=period, min_periods=period).apply(cal_sharpe)
        
    for period in [60, 120, 200, 250]:        
        data_df[f'{name_pref}_zscore{period}'] = data_df[feat].rolling(window=period, min_periods=period).apply(cal_zscore)
        

In [ ]:
#yfinance_tickers, fed_tickers
for feat in final_feats:
    if (feat in yfinance_tickers) | (feat in fed_tickers):
        print('Raw feat: ', feat)
        continue
    elif '_zscore' in feat:
        iloc=feat.index('zscore')+len('zscore')
        period = int(feat[iloc:])
        raw_feat = feat.split('_')[0]
        df[feat] = df[raw_feat].rolling(window=period, min_periods=period).apply(cal_zscore)
    elif '_mean_std' in feat:
        iloc=feat.index('_mean_std')+len('_mean_std')
        period = int(feat[iloc:])
        raw_feat = feat.split('_')[0] 
        df[feat] = df[raw_feat].rolling(window=period, min_periods=period).apply(cal_sharpe)
    elif '_volat' in feat:
        iloc=feat.index('_volat')+len('_volat')
        period = int(feat[iloc:])
        raw_feat = feat.split('_')[0]
        df[feat] = cal_vol(df[raw_feat], period=period, eps=1e-8)
    elif '_ma_dist' in feat:
        iloc=feat.index('_ma_dist')+len('_ma_dist')
        pp_ = feat[iloc:].split('_')
        period1, period2 = int(pp_[0]), int(pp_[-1])
        raw_feat = feat.split('_')[0]        
        df[feat] = df[raw_feat].rolling(window=period1, min_periods=period1).mean() - df[raw_feat].rolling(window=period2, min_periods=period2).mean()
    elif '_above200dMA' in feat:
        iloc=feat.index('_chg')+len('_chg')
        period = int(feat[iloc:-len('_above200dMA')])
        raw_feat = feat.split('_')[0]
        if '_pct_chg' in feat:
            df[f'{raw_feat}_pct_chg{period}'] = df[raw_feat].pct_change(period)
            df[feat] = df[f'{raw_feat}_pct_chg{period}'].fillna(0).rolling(window=200, min_periods=200).apply(above_mean)
        else:
            df[f'{raw_feat}_chg{period}'] = df[raw_feat].diff(period)
            df[feat] = df[f'{raw_feat}_chg{period}'].fillna(0).rolling(window=200, min_periods=200).apply(above_mean)
    elif '_std' in feat:  
        if '_chg200' in feat:
            iloc=feat.index('_std')+len('_std')
            period = int(feat[iloc:-len('_chg200')])
            raw_feat = feat.split('_')[0]

            df[f'{raw_feat}_std{period}'] = df[raw_feat].rolling(window=period, min_periods=period).std()
            df[feat] = df[f'{raw_feat}_std{period}'].diff(200)
        else:
            iloc=feat.index('_std')+len('_std')
            period = int(feat[iloc:])
            raw_feat = feat.split('_')[0]
            
            df[feat]  = df[raw_feat].rolling(window=period, min_periods=period).std()
    elif '_chg' in feat:
        if ('_above200dMA' in feat) | ('_std' in feat):
            continue
        else:
            if '_pct_chg' in feat:
                iloc=feat.index('_pct_chg')+len('_pct_chg')
                period = int(feat[iloc:])
                raw_feat = feat.split('_')[0]
                
                df[feat] = df[raw_feat].pct_change(period)
            else:
                
                iloc=feat.index('_chg')+len('_chg')
                period = int(feat[iloc:])
                raw_feat = feat.split('_')[0]
                
                df[feat] = df[raw_feat].diff(period)
    else:
        print('No category: ', feat)
                
                
        

<p style="font-family: 'Mochiy Pop P One';font-size:20px; font-weight: bold; color:black"><mark>!!!! Make sure all features in final_feats list in df </mark></p>

In [ ]:
set(final_feats)-set(df.columns)


<p style="font-family: 'Mochiy Pop P One';font-size:20px; font-weight: bold; color:mediumblue">Impute missing values</p>


In [ ]:
df.replace([-np.inf, np.inf], np.nan, inplace=True)
df.isna().sum()[df.isna().sum()>0].sort_values()

In [ ]:
df.tail(5)

In [ ]:
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

In [ ]:
float64_cols = df.select_dtypes(include=['float64']).columns.tolist()
df[float64_cols] = np.float32(df[float64_cols].values)
#---convert int64 to int32
int64_cols = df.select_dtypes(include=['int64']).columns.tolist()
df[int64_cols] = np.int32(df[int64_cols].values)

#-- save to parquet file
#pq.write_table(pa.Table.from_pandas(df), 'data.parquet', compression = 'GZIP')

<a id="4"></a>

<p style="font-family: 'Mochiy Pop P One';font-size:20px;  font-weight: bold; color:teal">4. Train LightGBM Models  </p>


<p style="font-family: 'Mochiy Pop P One';font-size:14px;  font-weight: bold; color:forestgreen; font-style:italic" id="toc">Quick Navigation Of This Notebook: </p>
<div align="left" style="font-family: 'Mochiy Pop P One';font-size:13px; font-weight: bold; color: teal">
<span style="padding:10px;background:wheat;margin-left:2px"> <a href=#top stype="color: teal">BACK TO TOP</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"> <a href=#1 stype="color: teal">1. Install and Load Packages</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#2 stype="color: teal">2. Data Scraping and Processing</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#3 stype="color: teal">3. Feature Engineering</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#4 stype="color: teal">4. ...</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#5 stype="color: teal">5. Predict</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#6 stype="color: teal">6. Today's Prediction</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#7 stype="color: teal">7. Explainable ML</a> </span>
</div>


In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import json

def get_tree_preds(X_train, y_train, X_test, params,
                   num_round=2000, cust_eval=None):

    dtrain = lgb.Dataset(X_train, y_train)
    params['verbosity'] = -1
    i_model = lgb.train(params = params,
                        train_set = dtrain,
                        num_boost_round=num_round,
                        feval=cust_eval,
                        early_stopping_rounds=None)

    y_preds = i_model.predict(X_test, num_iteration=i_model.best_iteration)
    scores = i_model.feature_importance(importance_type='gain', iteration=i_model.best_iteration)
    df_scores = pd.DataFrame({'feature':list(X_train.columns), 'gain': list(scores)})

    return y_preds, df_scores, i_model

In [ ]:
#final set of hyperparameters from: 
#https://www.kaggle.com/code/xxxxyyyy80008/equity-market-crisis-regime-prediction-using-gbdt7#2.-LightGBM

params_dict = {
0: {'bagging_fraction': 0.435, 'bagging_freq': 8, 'boosting': 'gbdt', 'feature_fraction': 0.4, 'lambda_l1': 1.473, 'lambda_l2': 0.994, 'learning_rate': 0.02, 'max_bin': 121, 'max_depth': 2, 'metric': 'auc', 'min_data_in_leaf': 75, 'min_gain_to_split': 0.41, 'min_sum_hessian_in_leaf': 1.21, 'num_boost_round': 870, 'objective': 'binary', 'scale_pos_weight': 20, 'seed': 1},
1: {'bagging_fraction': 0.485, 'bagging_freq': 1, 'boosting': 'gbdt', 'feature_fraction': 0.4, 'lambda_l1': 0.594, 'lambda_l2': 0.379, 'learning_rate': 0.02, 'max_bin': 306, 'max_depth': 2, 'metric': 'auc', 'min_data_in_leaf': 100, 'min_gain_to_split': 0.09, 'min_sum_hessian_in_leaf': 0.83, 'num_boost_round': 810, 'objective': 'binary', 'scale_pos_weight': 20, 'seed': 1},
2: {'bagging_fraction': 0.435, 'bagging_freq': 9, 'boosting': 'gbdt', 'feature_fraction': 0.6, 'lambda_l1': 1.047, 'lambda_l2': 0.866, 'learning_rate': 0.02, 'max_bin': 420, 'max_depth': 2, 'metric': 'auc', 'min_data_in_leaf': 95, 'min_gain_to_split': 0.38, 'min_sum_hessian_in_leaf': 1.04, 'num_boost_round': 750, 'objective': 'binary', 'scale_pos_weight': 20, 'seed': 1},
3: {'bagging_fraction': 0.545, 'bagging_freq': 11, 'boosting': 'gbdt', 'feature_fraction': 0.5, 'lambda_l1': 1.797, 'lambda_l2': 0.371, 'learning_rate': 0.12, 'max_bin': 338, 'max_depth': 1, 'metric': 'auc', 'min_data_in_leaf': 100, 'min_gain_to_split': 0.42, 'min_sum_hessian_in_leaf': 1.2, 'num_boost_round': 880, 'objective': 'binary', 'scale_pos_weight': 20, 'seed': 1},
4: {'bagging_fraction': 0.445, 'bagging_freq': 19, 'boosting': 'gbdt', 'feature_fraction': 0.7, 'lambda_l1': 1.307, 'lambda_l2': 0.296, 'learning_rate': 0.02, 'max_bin': 239, 'max_depth': 1, 'metric': 'auc', 'min_data_in_leaf': 90, 'min_gain_to_split': 0.15, 'min_sum_hessian_in_leaf': 0.4, 'num_boost_round': 880, 'objective': 'binary', 'scale_pos_weight': 20, 'seed': 1},
    
}

In [ ]:
yr = datetime.now().year
yr

In [ ]:
if yr%2==0: #2022, 2024
    train_loc = (df.index<f'{yr-1}-12-01') & (df.index>=f'{yr-15}-12-01') 
    test_loc = (df.index>=f'{yr}-01-01') & (df.index<f'{yr+2}-01-01')    
else: #2023
    train_loc = (df.index<f'{yr-2}-12-01') & (df.index>=f'{yr-16}-12-01') 
    test_loc = (df.index>=f'{yr-1}-01-01') & (df.index<f'{yr+1}-01-01')   

kfold_list = []
kfold_list.append([df.loc[train_loc, final_feats], df.loc[train_loc, 'target'],
                    df.loc[test_loc, final_feats], df.loc[test_loc, ['target']],
                      ]
                     )
for i, (a, b, c, d) in enumerate(kfold_list):
    print(i+1, a.shape, b.shape, c.shape, d.shape, a.index.min(), a.index.max(), c.index.min(), c.index.max())

In [ ]:
score_dfs = []
preds_list = []
pred_cols = []
models_dict = {}
for iter_i, params in params_dict.items():
    params = copy.deepcopy(params)
    print(params)
    
    num_boost_round = params['num_boost_round']
    del params['num_boost_round']

    scores_i = []
    preds_list_i = []
    models_list_i = []
    for j, (X_train, y_train, X_test, y_test) in enumerate(kfold_list):
        y_preds, df_scores, i_model = get_tree_preds(X_train, y_train, X_test, params=params,
                                                       num_round=num_boost_round, 
                                                       cust_eval=None)
        df_pred_i = y_test.copy(deep=True)
        df_pred_i[f'pred{iter_i}'] = y_preds
        scores_i.append(df_scores)
        preds_list_i.append(df_pred_i)
        models_list_i.append(i_model)
    
    models_dict[iter_i] = models_list_i
        
    
    df_pred = pd.concat(preds_list_i, axis=0)
                 
    pred_cols.append(f'pred{iter_i}')
    preds_list.append(df_pred)

In [ ]:
t = pd.concat(preds_list, axis=1)
df_pred = t.iloc[:, :1].copy(deep=True)
df_pred[pred_cols] = t[pred_cols]
df_pred.head(5)

In [ ]:
df_pred.tail(5)

<a id="5"></a>

<p style="font-family: 'Mochiy Pop P One';font-size:20px;  font-weight: bold; color:teal">5. Predict   </p>


<p style="font-family: 'Mochiy Pop P One';font-size:14px;  font-weight: bold; color:forestgreen; font-style:italic" id="toc">Quick Navigation Of This Notebook: </p>
<div align="left" style="font-family: 'Mochiy Pop P One';font-size:13px; font-weight: bold; color: teal">
<span style="padding:10px;background:wheat;margin-left:2px"> <a href=#top stype="color: teal">BACK TO TOP</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"> <a href=#1 stype="color: teal">1. Install and Load Packages</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#2 stype="color: teal">2. Data Scraping and Processing</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#3 stype="color: teal">3. Feature Engineering</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#4 stype="color: teal">4. Train LightGBM Models</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#5 stype="color: teal">5. ...</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#6 stype="color: teal">6. Today's Prediction</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#7 stype="color: teal">7. Explainable ML</a> </span>
</div>


In [ ]:
df_pred['pred_mean']=df_pred[pred_cols].mean(axis=1)
df_pred['pred_mean_label']=((df_pred[pred_cols]>0.5).mean(axis=1)>0.5).astype(int)

In [ ]:
df_pred['target']= df_pred['target'].values.tolist()[:-3]+[np.nan]*3

In [ ]:
df_pred[pred_cols+['target']].plot(figsize=(18, 6), secondary_y=['target'], grid=True)
df_pred[['target', 'pred_mean',]].plot(figsize=(18, 6), secondary_y=['target'], grid=True)
df_pred[['target', 'pred_mean_label']].plot(figsize=(18, 6),  grid=True)

<a id="6"></a>

<p style="font-family: 'Mochiy Pop P One';font-size:20px;  font-weight: bold; color:teal">6. Today's Prediction  </p>


<p style="font-family: 'Mochiy Pop P One';font-size:14px;  font-weight: bold; color:forestgreen; font-style:italic" id="toc">Quick Navigation Of This Notebook: </p>
<div align="left" style="font-family: 'Mochiy Pop P One';font-size:13px; font-weight: bold; color: teal">
<span style="padding:10px;background:wheat;margin-left:2px"> <a href=#top stype="color: teal">BACK TO TOP</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"> <a href=#1 stype="color: teal">1. Install and Load Packages</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#2 stype="color: teal">2. Data Scraping and Processing</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#3 stype="color: teal">3. Feature Engineering</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#4 stype="color: teal">4. Train LightGBM Models</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#5 stype="color: teal">5. Predict</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#6 stype="color: teal">6. ...</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#7 stype="color: teal">7. Explainable ML</a> </span>
</div>


In [ ]:
df_pred.tail(10)

<a id="7"></a>

<p style="font-family: 'Mochiy Pop P One';font-size:20px;  font-weight: bold; color:teal">7. Explainable ML </p>


<p style="font-family: 'Mochiy Pop P One';font-size:14px;  font-weight: bold; color:forestgreen; font-style:italic" id="toc">Quick Navigation Of This Notebook: </p>
<div align="left" style="font-family: 'Mochiy Pop P One';font-size:13px; font-weight: bold; color: teal">
<span style="padding:10px;background:wheat;margin-left:2px"> <a href=#top stype="color: teal">BACK TO TOP</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"> <a href=#1 stype="color: teal">1. Install and Load Packages</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#2 stype="color: teal">2. Data Scraping and Processing</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#3 stype="color: teal">3. Feature Engineering</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#4 stype="color: teal">4. Train LightGBM Models</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#5 stype="color: teal">5. Predict</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#6 stype="color: teal">6. Today's Prediction</a> </span>
<span style="padding:10px;background:wheat;margin-left:2px"><a href=#7 stype="color: teal">7. ... </a> </span>
</div>


In [ ]:
import shap

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import statsmodels.api as sm
import seaborn as sns

In [ ]:
last_sample = df_pred.index[-1]
X_test = kfold_list[0][-2]
features = X_test.loc[X_test.index==last_sample] #examine one date. this is a data frame

In [ ]:
for iter_i, v in models_dict.items():
    tree_model = v[0]
    explainer = shap.TreeExplainer(tree_model, num_iteration=tree_model.best_iteration)

    

    shap_values = explainer.shap_values(features)[1] #shap values for the row
    shap_interaction_values = explainer.shap_interaction_values(features)

    if isinstance(shap_interaction_values, list):
        shap_interaction_values = shap_interaction_values[1]
    dis = [f'{a} ({b:.03f})' for a, b in zip(features.columns, features.values[0].round(3))]
    shap_bar = pd.DataFrame(data={'feat': dis, 'shap': shap_values[0]})
    x_ = shap_bar['shap']
    shap_bar['shap_z'] = (x_ - x_.mean())/x_.std()
    shap_bar['colors'] = ['red' if x_ > 0 else 'green' for x_ in shap_bar['shap_z']]
    shap_bar = shap_bar[shap_bar['shap_z'].abs()>0.4]
    shap_bar.sort_values('shap_z', inplace=True, ascending=False)
    shap_bar.reset_index(inplace=True)
    # Draw plot
    plt.figure(figsize=(18,20), dpi= 100)
    plt.hlines(y=shap_bar.index, xmin=0, xmax=shap_bar.shap_z, color=shap_bar.colors, alpha=0.9, linewidth=6)
    for x, y, tex in zip(shap_bar.shap_z, shap_bar.index, shap_bar.shap_z):
        t = plt.text(x, y, round(tex, 2), horizontalalignment='right' if x < 0 else 'left',
                     verticalalignment='center', fontdict={'color':'red' if x > 0 else 'green', 'size':14})

    # Decorations
    plt.yticks(shap_bar.index, shap_bar.feat, fontsize=12)
    plt.title(f"SHAP of model {iter_i+1}: Normalized contribution of features to crash probability ({last_sample.strftime('%Y-%m-%d')}, \
    prob: {df_pred.loc[last_sample]['pred_mean']:.03f})", 
              fontdict={'size':18})
    plt.grid(linestyle='--', alpha=0.5)
    # plt.xlim(-2.5, 2.5)
    plt.show()    